## Задание

В этом задании требуется сравнить различные **меры близости** объектов при помощи тезауруса WordNet

По [ссылке](https://courses.openedu.ru/assets/courseware/v1/e585af30b55a89a85efcabc6a6ac0bfa/asset-v1:ITMOUniversity+ADVMLAUTXT+spring_2023_ITMO_mag+type@asset+block/Task_4_sample_4.csv) представлена выборка из датасета **WordSim353**. Ваша задача, используя тезаурус WordNet, вычислить оценки близости для всех элементов соответствующих синсетов представленных пар слов.

В качестве близости двух слов использовать наибольшую близость среди соответствующих элементов рассматриваемых синсетов.

В качестве меры близости использовать: близость на основе пути (`path_similarity`), меру Leacock-Chodorow (`lch_similarity`) и меру Wu-Palmer (`wup_similarity`).

Вычислить **коэффициент ранговой корреляции Спирмена** для каждой меры близости, используя известную экспертную оценку (колонка `Score` в выборке).

## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [2]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [3]:
import pandas as pd

url = "https://courses.openedu.ru/assets/courseware/v1/e585af30b55a89a85efcabc6a6ac0bfa/asset-v1:ITMOUniversity+ADVMLAUTXT+spring_2023_ITMO_mag+type@asset+block/Task_4_sample_4.csv"

triples = pd.read_csv(url, encoding='utf-8')
triples.head()

,word_1,word_2,Score
0,professor,cucumber,0.31
1,monk,slave,0.92
2,psychology,discipline,5.58
3,life,death,7.88
4,announcement,production,3.38


In [4]:
score_map = {tuple(triple[:2]): float(triple.Score) for id, triple in triples.iterrows()}
len(score_map)

150

Отметим, что в шаблоне решения вместо исходного набора данных использовали только его часть с экспертными оценками сходства (`similarity`) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [5]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: professor-cucumber
Ground truth score: 0.31

Path: 0.077 
wup: 0.500 
shortest_path: 12.000

Words: monk-slave
Ground truth score: 0.92

Path: 0.200 
wup: 0.667 
shortest_path: 4.000


Вычисляем для всех пар несколько оценок

In [15]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list, lch_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos='n')
    all_w2 = wn.synsets(w2, pos='n')
    
    # добавляем интересующие нас метрики и экспертные оценки
    
    # add Leacock-Chodorow metric
    lch = max([item1.lch_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(lch)

    wup = max([item1.wup_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

In [16]:
len(wup_list)

150

## Вычисляем ранговую корреляцию Спирмена

In [20]:
from scipy.stats import spearmanr

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch Spearman R: %.4f" % coef)

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

path Spearman R: 0.6535
lch Spearman R: 0.6535
wup  Spearman R: 0.6936


При помощи метода `hyponyms()` найдите количество гипонимов для синсета `wood.n.01`, а также при помощи метода `name()` найдите значение первого в гипонима из списка.

In [18]:
hyponyms = wn.synset('wood.n.01').hyponyms()
len(hyponyms)

91

Первый элемент списка гипонимов синсета `wood.n.01`

In [19]:
hyponyms[0].name()

'alder.n.01'